In [1]:
import pandas as pd
import csv

name = 'thairath'
#path = '../crawl_data/' + name + '.csv'
user_sim_out = 'user_similarity_' + name+ '.csv'

#data = pd.read_csv(path, encoding="utf8") 
#data.shape
#data.head()

In [21]:
import csv

path = '../crawl_data/post interaction/' + name + '_post_interact.csv'

data = pd.read_csv(path, encoding="utf8") 
#data.shape
data.head()

#love = data.loc[data['type'] == 'Love']
#love.head()

#post_url = data.loc[data['post_url'] == 'https://www.facebook.com/khaosod/posts/4333442143339378']
#post_url.size
#post_url.head()

#user_url = data.loc[data['user_url'] == '/profile.php?id=100043671036759']
#user_url.head()

#data.size

,Unnamed: 0,post_url,react_url,type,user_url,usr_name,crawl_timestamp
0,0,https://www.facebook.com/thairath/posts/101586...,/ufi/reaction/profile/browser/fetch/?limit=400...,Wow,/profile.php?id=100043772795365,เตย แล้วไง,2019-11-16 10:27:30.369146
1,1,https://www.facebook.com/thairath/posts/101586...,/ufi/reaction/profile/browser/fetch/?limit=400...,Wow,/kanaput.pituksakilcuy,Kanaput Pituksakilcuy,2019-11-16 10:27:30.369146
2,2,https://www.facebook.com/thairath/posts/101586...,/ufi/reaction/profile/browser/fetch/?limit=400...,Wow,/jzbs.sha,Jzbs Sha,2019-11-16 10:27:30.369146
3,3,https://www.facebook.com/thairath/posts/101586...,/ufi/reaction/profile/browser/fetch/?limit=400...,Wow,/nantiporn.chalee,Nantiporn Chalee,2019-11-16 10:27:30.369146
4,4,https://www.facebook.com/thairath/posts/101586...,/ufi/reaction/profile/browser/fetch/?limit=400...,Wow,/peek.phachariya,Peek Phachariya,2019-11-16 10:27:30.369146


In [3]:
from tqdm import tqdm, tqdm_notebook

'''
    map[user_url][post_user_type] = user_url
'''
mapper = {}

print ("row: %d" % (data.size))

for row in tqdm_notebook(data.iterrows()):
    
    post_url = row[1]['post_url']
    type_t = row[1]['type']
    user_url = row[1]['user_url']

    if not post_url in mapper:
        mapper[post_url] = {}

    if not type_t in mapper[post_url]:
        mapper[post_url][type_t] = []
    
    mapper[post_url][type_t].append(user_url)
        
print ("Finished!")

row: 1680105



Finished!


In [4]:
# Create relation between users
user_map = {}

for post_url, post_data in tqdm_notebook(mapper.items()):
    #print (post_url)
    for type_k, type_data in post_data.items():
        #print (type_k)
        #print (len(type_data))
        #print (type_data)
        for user_id_i in type_data:
            for user_id_j in type_data:
                if not user_id_i in user_map:
                    user_map[user_id_i] = {}
                if not user_id_j in user_map[user_id_i]:
                    user_map[user_id_i][user_id_j] = 0
                    
                user_map[user_id_i][user_id_j] += 1

In [5]:
# Write to file to make snapshot (The file is very huge)

#w = csv.writer(open("user_map.csv", "w"))
#for key, val in tqdm_notebook(user_map.items()):
#    w.writerow([key, val])

In [6]:
# Maybe, this needs csv reader to dict

In [7]:
from tqdm import tqdm, tqdm_notebook

# Get similarity set
out_data = []

for user_i, user_i_dat in tqdm_notebook(user_map.items()):
    i_action_count = len(user_map[user_i])
    
    if i_action_count < 10:
        continue
        
    for user_j in user_map[user_i]:
        
        if user_i == user_j:
            continue
            
        j_action_count = len(user_map[user_j])
        if j_action_count < 10:
            continue
        
        ij_action_count = user_map[user_i][user_j]
        if ij_action_count < 10:
            continue
        
        row = []
        row.append(user_i)
        row.append(user_j)
        row.append(ij_action_count)
        i_not_ij = i_action_count - ij_action_count
        row.append(i_not_ij)
        j_not_ij = j_action_count - ij_action_count
        row.append(j_not_ij)
        
        # Get jaccard similarity
        intersec = ij_action_count
        union = i_not_ij + ij_action_count + j_not_ij
        row.append(intersec / union)
        
        # Get overlapping coefficient
        min_ij = min((i_not_ij, j_not_ij))
        #print ('min_ij: %d' % min_ij)
        row.append(intersec / min_ij)
        
        out_data.append(row)
    
df = pd.DataFrame(out_data, columns = ["user_i", "user_j", "intersec", "i_not_intersec", "j_not_intersec", "jaccard_sim", "overlap_coef"])
df.head()

,user_i,user_j,intersec,i_not_intersec,j_not_intersec,jaccard_sim,overlap_coef
0,/orathai.inpan,/profile.php?id=100023725447664,10,10706,34636,0.000220,0.000934
1,/orathai.inpan,/kamoltip.lengpanich.52,10,10706,71631,0.000121,0.000934
2,/orathai.inpan,/niyomsaeiiw,12,10704,90180,0.000119,0.001121
3,/orathai.inpan,/profile.php?id=100034314500681,12,10704,56350,0.000179,0.001121
4,/orathai.inpan,/poopae.poopae.9889,12,10704,71749,0.000146,0.001121


In [8]:
with open(user_sim_out, 'w') as csvfile:
    
    # Create file writer
    w = csv.writer(csvfile, delimiter=',')
    
    # Write header
    h = ["user_i", "user_j", "intersec", "i_not_intersec", "j_not_intersec", "jaccard_sim", "overlap_coef"]
    w.writerow(h)
    
    # Write row
    for i in tqdm_notebook(range(len(out_data))):
        row = out_data[i]
        w.writerow(row)

In [9]:
# Might need a file reader for user similarity

In [10]:
df = pd.DataFrame(out_data, columns = ["user_i", "user_j", "intersec", "i_not_intersec", "j_not_intersec", "jaccard_sim", "overlap_coef"])
df.head()

,user_i,user_j,intersec,i_not_intersec,j_not_intersec,jaccard_sim,overlap_coef
0,/orathai.inpan,/profile.php?id=100023725447664,10,10706,34636,0.000220,0.000934
1,/orathai.inpan,/kamoltip.lengpanich.52,10,10706,71631,0.000121,0.000934
2,/orathai.inpan,/niyomsaeiiw,12,10704,90180,0.000119,0.001121
3,/orathai.inpan,/profile.php?id=100034314500681,12,10704,56350,0.000179,0.001121
4,/orathai.inpan,/poopae.poopae.9889,12,10704,71749,0.000146,0.001121


In [11]:
#sort
df_by_overlap_coef = df.sort_values(['overlap_coef'], ascending=[0])
df_by_overlap_coef.head()

,user_i,user_j,intersec,i_not_intersec,j_not_intersec,jaccard_sim,overlap_coef
31680,/profile.php?id=100025444880233,/profile.php?id=100015601411398,12,2448,1837,0.002793,0.006532
71373,/profile.php?id=100015601411398,/profile.php?id=100025444880233,12,1837,2448,0.002793,0.006532
71374,/profile.php?id=100015601411398,/profile.php?id=100023725447664,10,1839,34636,0.000274,0.005438
179,/profile.php?id=100023725447664,/profile.php?id=100015601411398,10,34636,1839,0.000274,0.005438
31681,/profile.php?id=100025444880233,/wongchai.seangram,12,2448,22396,0.000483,0.004902


In [48]:
# Find intersection activity

intersec_action_list = []

target_user_i = "/profile.php?id=100025444880233"
target_user_j = "/wongchai.seangram"

for post_url, post_data in tqdm_notebook(mapper.items()):
    for type_k, type_data in post_data.items():
        for user_id_i in type_data:
            for user_id_j in type_data:
                if user_id_i == target_user_i and user_id_j == target_user_j:
                    row = []
                    row.append(post_url)
                    row.append(type_k)
                    #row.append(user_id_i)
                    #row.append(user_id_j)
                    row.append("")
                    
                    intersec_action_list.append(row)

In [51]:
df = pd.DataFrame(intersec_action_list, columns = ["post_url", "reaction", "message"])
df.head(20)

,post_url,type_data,message
0,https://www.facebook.com/thairath/posts/101586...,Sad,
1,https://www.facebook.com/thairath/posts/101586...,Wow,
2,https://www.facebook.com/thairath/photos/a.301...,Wow,
3,https://www.facebook.com/thairath/posts/101586...,Haha,
4,https://www.facebook.com/thairath/videos/82532...,Wow,
5,https://www.facebook.com/thairath/posts/101586...,Wow,
6,https://www.facebook.com/thairath/posts/101586...,Love,
7,https://www.facebook.com/thairath/posts/101586...,Haha,
8,https://www.facebook.com/thairath/videos/24307...,Wow,
9,https://www.facebook.com/thairath/posts/101586...,Sad,


In [50]:
path = '../crawl_data/' + name + '.csv'

data = pd.read_csv(path, encoding="utf8") 
data.head()

,Unnamed: 0,post_page_id,post_id,post_time,post_link,post_message,post_like_count,post_share_count,post_reactor_count,post_comment_count,post_angry_count,post_love_count,post_sad_count,post_wow_count,post_haha_count,post_crawl_time,page_name
0,0,146406732438,10158670077292439,2019-11-14 23:32:53,https://www.facebook.com/thairath/posts/101586...,"ไทยตอนบนอากาศเย็น-ลมแรง มีฝนบางแห่งบริเวณ ""ภาค...",3404,521,3463,26,0,1,3,37,18,2019-11-15 01:05:39.503125,thairath
1,1,146406732438,10158670062507439,2019-11-14 23:27:08,https://www.facebook.com/thairath/posts/101586...,อินโดนีเซียยกเลิกเตือนภัยสึนามิแล้ว หลังเกิดแผ...,286,7,294,1,0,0,4,4,0,2019-11-15 01:05:39.503125,thairath
2,2,146406732438,10158670036642439,2019-11-14 23:19:12,https://www.facebook.com/thairath/posts/101586...,"ตัดเกรด ""ทีมชาติไทย"" หลังเกมพ่าย ""ทีมชาติมาเลเ...",362,2,403,146,2,3,8,1,27,2019-11-15 01:05:39.503125,thairath
3,3,146406732438,10158669947832439,2019-11-14 22:42:21,https://www.facebook.com/thairath/posts/101586...,อีก 500 ม.จะถึงบ้าน สลดหนุ่ม พนง.ขนส่ง ควบบิ๊ก...,1004,109,1205,46,1,1,176,14,9,2019-11-15 01:05:39.503125,thairath
4,4,146406732438,10158669940462439,2019-11-14 22:39:25,https://www.facebook.com/thairath/posts/101586...,"ลงทะเบียน ""ชิมช้อปใช้ เฟส 3"" วันที่ 2 รอบแรก 6...",207,15,252,70,9,0,3,4,29,2019-11-15 01:05:39.503125,thairath


In [73]:
for row in tqdm_notebook(df.iterrows()):
    post_url = row[1]['post_url']
    sel = data.loc[data['post_link'] == post_url]
    post_message = sel['post_message']
    row[1]['message'] = post_message
    
df.head(20)

,post_url,type_data,message
0,https://www.facebook.com/thairath/posts/101586...,Sad,5 หนุ่มใหญ่ขับเก๋ง ชนท้ายพ่วงดับคาซาก คนขับ...
1,https://www.facebook.com/thairath/posts/101586...,Wow,8 หาดูยาก อุทยานแห่งชาติแก่งกระจาน เผยภาพเส...
2,https://www.facebook.com/thairath/photos/a.301...,Wow,29 จบเกม ทีมชาติมาเลเซีย แซงกลับมาเฉือนชนะท...
3,https://www.facebook.com/thairath/posts/101586...,Haha,45 จบด้วยดี ลุงขับกระบะขวางรถฉุกเฉินเปิดไซเ...
4,https://www.facebook.com/thairath/videos/82532...,Wow,145 ตำรวจปราบปรามยาเสพติด จับกุมผู้ต้องพร้อ...
5,https://www.facebook.com/thairath/posts/101586...,Wow,151 ตำรวจ ปส.บุกโกดังจับผู้ต้องหาพร้อมยึดลู...
6,https://www.facebook.com/thairath/posts/101586...,Love,154 ทหารฮีโร่ เห็นนักเรียน ม.5 กระโดดสะพานแ...
7,https://www.facebook.com/thairath/posts/101586...,Haha,"160 ขาประจำ""บิ๊กตู่"" ""เสี่ยแดง""ย้ำ รีบเปลีย..."
8,https://www.facebook.com/thairath/videos/24307...,Wow,193 ตำรวจปราบปรามยาเสพติด บุกจับแก๊งขนไอซ์ม...
9,https://www.facebook.com/thairath/posts/101586...,Sad,196 รถตู้โดยสารส่งต่างด้าวจำนวน 15 รายพลิกค...
